# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [10]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [11]:
import urllib2
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [12]:
## YOUR CODE HERE
#resultIndeed = urllib2.urlopen(URL).read()
url = "http://www.indeed.com/jobs?q=data+scientist+%2490%2C000+-+%24200%2C000&l={}&start={}"


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [13]:
## YOUR CODE HERE
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
    
def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [14]:
rows = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Atlanta']):
    for start in range(1, 1000, 10):
        r = requests.get(url.format(city, start))
        soup = BeautifulSoup(r.content,"lxml")
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['salary'] = get_salary_from_result(result)
                row['city'] = city
                rows.append(row)



#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [15]:
## Code completed above
## YOUR CODE HERE
data = pd.DataFrame.from_records(rows)

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [16]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [17]:
## YOUR CODE HERE

### Save your results as a CSV

In [18]:
## YOUR CODE HERE
data.to_csv("../assets/indeedproject.csv", index=False,encoding='utf-8')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [19]:
## YOUR CODE HERE
import pandas as pd
salary_data = pd.read_csv("../assets/indeedproject.csv")
salary_data = salary_data[salary_data.salary.notnull()]
salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('hour'))]

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [20]:
## YOUR CODE HERE
import numpy as np 
def cleanSalaryColumn(salary):
    badChars = ['a year','a month', " ", "$", "a",",","dy"]
    string = salary
    for x in badChars:
        string = string.replace(x,"")
    return string

def findBound(salary,boundIndex):
    if (salary == np.nan) | (salary == None):
        return salary
    else:
        if  "-" in salary:
            return salary.split("-")[boundIndex]
        return salary

salary_data['salary_as_num'] = salary_data.salary.apply(cleanSalaryColumn)
salary_data['salary_ub'] = salary_data.salary_as_num.apply( lambda x: findBound(x,1) ).astype(float)
salary_data['salary_lb'] = salary_data.salary_as_num.apply( lambda x: findBound(x,0) ).astype(float)


`MG: Good job below`

In [21]:
def getClass(x,threshold):
    ub = x[6]
    lb = x[7]
    sal = x[2]
    if 'month' in sal.lower():
        ub = float(ub * 12)
        lb = float(lb * 12)
        
    avg = float(( ub + lb ) / 2)
    if avg >= threshold:
        return 1
    else:
        return 0
    
def binOccupation(title):
    title = title.lower()
    
#    finalBin = 0
    if 'scientist' in title:
        finalBin =  1
    elif 'engineer' in title:
        finalBin =  2
    elif 'product' in title:
        finalBin =  3
    elif 'stat' in title:
        finalBin =  4
    elif 'analyst' in title:
        finalBin =  5
    elif ('software engineer' in title) | ('developer' in title):
        finalBin =  6
    else:
        finalBin = 0
   
    return finalBin

def binSeniority(title):
    title = title.lower()
  
#    finalBin = 0
    if ('sr' in title) | ('senior' in title):
        finalBin = 1
    elif ('jr' in title) | ('junior' in title):
        finalBin = 2
    elif ('mgr' in title) | ('manager' in title):
        finalBin = 2
    elif ('associate' in title) | ('ass' in title) | ('mid' in title):
        finalBin = 3
    elif 'vp' in title:
        finalBin = 4
    elif 'lead' in title:
        finalBin = 5
    elif 'director' in title:
        finalBin = 6
    else:
        finalBin = 0
    return finalBin
        
#def monthSalYear(x):
    
salary_data['occupationGrp'] = salary_data.title.apply(binOccupation).astype(int)
salary_data['seniorityGrp'] = salary_data.title.apply(binSeniority).astype(int)
salary_data['class'] = salary_data.apply( lambda x: getClass(x,130000), axis = 1 ).astype(int)
salary_data

city                             company                      salary  \
4     Chicago                    All-In Analytics  $115,000 - $150,000 a year   
12    Chicago               Workbridge Associates   $80,000 - $110,000 a year   
35    Chicago  Seeking Alpha Search International  $110,000 - $200,000 a year   
52    Chicago               Workbridge Associates  $120,000 - $160,000 a year   
54    Chicago                            Hirewell             $125,000 a year   
69    Chicago               Workbridge Associates  $140,000 - $180,000 a year   
71    Chicago               Workbridge Associates  $100,000 - $140,000 a year   
78    Chicago                         CREhire.com             $138,300 a year   
98    Chicago                 Analytic Recruiting             $140,000 a year   
107   Chicago                  Jobspring Partners  $100,000 - $120,000 a year   
109   Chicago             Smith Hanley Associates   $80,000 - $100,000 a year   
110   Chicago                              3coast   $70,000 - $110,000 a year   
113   Chicago             Smith Hanley Associates  $150,000 - $200,000 a year   
116   Chicago               Workbridge Associates             $185,000 a year   
130   Chicago          North shore Staffing Group              $95,000 a year   
145   Chicago               Workbridge Associates             $165,000 a year   
161   Chicago                            Hirewell             $120,000 a year   
168   Chicago                 Analytic Recruiting  $100,000 - $140,000 a year   
188   Chicago                          NJF Search             $170,000 a year   
198   Chicago                  Jobspring Partners   $80,000 - $100,000 a year   
232   Chicago               Workbridge Associates             $130,000 a year   
242   Chicago               Workbridge Associates             $180,000 a year   
259   Chicago                      Selby Jennings  $125,000 - $150,000 a year   
263   Chicago               Workbridge Associates  $110,000 - $150,000 a year   
282   Chicago               Workbridge Associates             $170,000 a year   
311   Chicago                  Jobspring Partners   $60,000 - $120,000 a year   
559   Chicago                  Jobspring Partners   $60,000 - $120,000 a year   
573   Chicago                  Jobspring Partners   $60,000 - $120,000 a year   
588   Chicago                  Jobspring Partners   $60,000 - $120,000 a year   
603   Chicago                  Jobspring Partners   $60,000 - $120,000 a year   
...       ...                                 ...                         ...   
8282   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8296   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8310   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8324   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8338   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8351   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8366   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8380   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8396   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8409   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8424   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8438   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8452   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8465   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8478   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8491   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8504   Austin       University of Texas at Austin    $6,250 - $10,833 a month   
8517   Austin       University of Texas 

In [22]:
colsToKeep = salary_data.columns[8:]
#print colsToKeep
cleanDf = salary_data[colsToKeep]
dummies1 = pd.get_dummies(cleanDf.occupationGrp, prefix = "occupation")
dummies2 = pd.get_dummies(cleanDf.seniorityGrp, prefix = "seniority")
join1 = cleanDf[ [cleanDf.columns[2]] ].join( dummies1 )
join2 = join1.join( dummies2  )
#print cleanDf.columns[2] 
join2

class  occupation_0  occupation_1  occupation_2  occupation_3  \
4         1           0.0           1.0           0.0           0.0   
12        0           0.0           1.0           0.0           0.0   
35        1           0.0           0.0           0.0           0.0   
52        1           0.0           1.0           0.0           0.0   
54        0           0.0           1.0           0.0           0.0   
69        1           0.0           1.0           0.0           0.0   
71        0           0.0           1.0           0.0           0.0   
78        1           0.0           1.0           0.0           0.0   
98        1           0.0           1.0           0.0           0.0   
107       0           0.0           1.0           0.0           0.0   
109       0           0.0           1.0           0.0           0.0   
110       0           0.0           0.0           0.0           1.0   
113       1           0.0           0.0           0.0           0.0   
116       1           1.0           0.0           0.0           0.0   
130       0           0.0           0.0           0.0           0.0   
145       1           0.0           1.0           0.0           0.0   
161       0           0.0           1.0           0.0           0.0   
168       0           1.0           0.0           0.0           0.0   
188       1           0.0           0.0           1.0           0.0   
198       0           0.0           0.0           1.0           0.0   
232       1           0.0           1.0           0.0           0.0   
242       1           1.0           0.0           0.0           0.0   
259       1           0.0           0.0           0.0           0.0   
263       1           0.0           1.0           0.0           0.0   
282       1           1.0           0.0           0.0           0.0   
311       0           0.0           0.0           1.0           0.0   
559       0           0.0           0.0           1.0           0.0   
573       0           0.0           0.0           1.0           0.0   
588       0           0.0           0.0           1.0           0.0   
603       0           0.0           0.0           1.0           0.0   
...     ...           ...           ...           ...           ...   
8282      0           0.0           1.0           0.0           0.0   
8296      0           0.0           1.0           0.0           0.0   
8310      0           0.0           1.0           0.0           0.0   
8324      0           0.0           1.0           0.0           0.0   
8338      0           0.0           1.0           0.0           0.0   
8351      0           0.0           1.0           0.0           0.0   
8366      0           0.0           1.0           0.0           0.0   
8380      0           0.0           1.0           0.0           0.0   
8396      0           0.0           1.0           0.0           0.0   
8409      0           0.0           1.0           0.0           0.0   
8424      0           0.0           1.0           0.0           0.0   
8438      0           0.0           1.0           0.0           0.0   
8452      0           0.0           1.0           0.0           0.0   
8465      0           0.0           1.0           0.0           0.0   
8478      0           0.0           1.0           0.0           0.0   
8491      0           0.0           1.0           0.0           0.0   
8504      0           0.0           1.0           0.0           0.0   
8517      0           0.0           1.0           0.0           0.0   
8530      0           0.0           1.0           0.0           0.0   
8543      0           0.0           1.0           0.0           0.0   
8556      0           0.0           1.0           0.0           0.0   
8569      0           0.0           1.0           0.0           0.0   
8582      0           0.0           1.0           0.0           0.0   
8595      0           0.0           1.0           0.0           0.0   
8608      0           0.0     

#### Thought experiment: What is the baseline accuracy for this model?

In [35]:
## YOUR CODE HERE
from sklearn.cross_validation import train_test_split
import statsmodels.formula.api as sm

featureCols = join2.columns[1:]
specialCol = join2.columns[:1]
X = join2[featureCols]
X = X[ X.columns[1:6] + X.columns[7:] ]
#y = join2['class']
y = join2[specialCol]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)
logit = sm.Logit(y_train, X_train)
result = logit.fit_regularized()
result.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.523551713179
            Iterations: 93
            Function evaluations: 94
            Gradient evaluations: 93


C:\Users\PASAGI\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                  207
Model:                          Logit   Df Residuals:                      195
Method:                           MLE   Df Model:                           11
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.2322
Time:                        10:17:53   Log-Likelihood:                -108.38
converged:                       True   LL-Null:                       -141.15
                                        LLR p-value:                 8.482e-10
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
occupation_1     0.9126      0.596      1.530      0.126        -0.256     2.081
occupation_2     1.0550      0.696      1.515      0.130        -0.310     2.420
occupation_3   -17.3749   8136.821     -0.002      0.998      -1.6e+04  1.59e+04
occupation_4    -0.3402      1.077     -0.316      0.752        -2.452     1.771
occupation_5     2.3362      0.738      3.167      0.002         0.890     3.782
seniority_0     -0.6334      0.486     -1.303      0.193        -1.586     0.320
seniority_1     -2.1592      0.631     -3.424      0.001        -3.395    -0.923
seniority_2     25.5875   2.99e+05   8.54e-05      1.000     -5.87e+05  5.87e+05
seniority_3     -3.1041      1.237     -2.510      0.012        -5.528    -0.681
seniority_4     34.2109   9.31e+06   3.68e-06      1.000     -1.82e+07  1.82e+07
seniority_5     -0.2195      1.362     -0.161      0.872        -2.889     2.450
seniority_6      0.5442      1.019      0.534      0.593        -1.454     2.542
================================================================================
"""

`MG: Where is your interpretation of the above results?`

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

`MG: You didn't do anything about location.`

In [34]:
## YOUR CODE HERE
logitO = sm.Logit(y, X)
resultO = logitO.fit_regularized()
resultO.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.540938874397
            Iterations: 101
            Function evaluations: 102
            Gradient evaluations: 101


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                  296
Model:                          Logit   Df Residuals:                      284
Method:                           MLE   Df Model:                           11
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.2091
Time:                        10:17:40   Log-Likelihood:                -160.12
converged:                       True   LL-Null:                       -202.46
                                        LLR p-value:                 1.820e-13
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
occupation_1     0.8796      0.472      1.864      0.062        -0.045     1.805
occupation_2     1.3322      0.587      2.271      0.023         0.182     2.482
occupation_3   -16.8519   5959.416     -0.003      0.998     -1.17e+04  1.17e+04
occupation_4     0.7410      0.881      0.841      0.400        -0.985     2.467
occupation_5     1.8063      0.581      3.108      0.002         0.667     2.945
seniority_0     -0.5335      0.396     -1.346      0.178        -1.310     0.243
seniority_1     -2.2532      0.505     -4.461      0.000        -3.243    -1.263
seniority_2     29.3470   1.27e+06    2.3e-05      1.000      -2.5e+06   2.5e+06
seniority_3     -2.9447      0.938     -3.138      0.002        -4.784    -1.105
seniority_4      1.3860      1.230      1.126      0.260        -1.025     3.797
seniority_5     -0.3179      0.989     -0.322      0.748        -2.256     1.620
seniority_6      0.7179      0.707      1.015      0.310        -0.668     2.104
================================================================================
"""

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [25]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [81]:
## YOUR CODE HERE
from sklearn.linear_model import LogisticRegression 
modelSk = LogisticRegression()
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)
print predictionSk

[1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0
 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1
 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [66]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)


predict    0   1
actual          
0        130  38
1         36  92

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [72]:
## YOUR CODE HERE
import sklearn.metrics as met

print met.accuracy_score(y,predictionSk)
print met.recall_score(y, predictionSk)
print met.precision_score(y, predictionSk)

0.75
0.71875
0.707692307692


#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [77]:
## YOUR CODE HERE
modelSk = LogisticRegression()
modelSk.penalty = "l1"
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)


In [78]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)


predict    0    1
actual           
0        112   56
1         28  100

In [79]:
## YOUR CODE HERE
modelSk = LogisticRegression()
modelSk.penalty = "l2"
resultSk = modelSk.fit(X, y.values.ravel())
predictionSk = resultSk.predict(X)


In [80]:
#print y
ravely = pd.Series(y.values.ravel())
ravely.name ="actual"
predictionSk = pd.Series(predictionSk)
predictionSk.name ="predict"
pd.crosstab(ravely, predictionSk)
#print len(y), len(predictionSk)

predict    0   1
actual          
0        130  38
1         36  92

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [30]:
## YOUR CODE HERE

In [31]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [32]:
## YOUR CODE HERE

Score: | 10/21
------|-------
Identify: Problem Statement / Aim| 2
Acquire: Import Data Using Requests + BeautifulSoup| 3
Parse: Clean & Organize Data| 2
Model: Perform Logistic Regression| 2
Evaluate: Logistic Regression				|1
Present: Write a report for your audience addressing findings & recommendations		| 0
Interactive visualization | 0
Bonus: Countvectorizer, Regularization Parameters| 0
Bonus! Present: Create blog post summary	| 0